In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:98% !important; }</style>"))

import pyodbc as pyodbc

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh import palettes
#from bokeh.palettes import Category20
#from bokeh.palettes import Accent
from bokeh.models import Range1d
import bokeh.models as bokeh_models
from bokeh.io import export_png

In [ ]:
## Save data
df = pd.read_pickle(r"dfEndocrinology.pkl")

In [ ]:
df['day'] = df['TimePoint'].dt.day.astype(int)

In [ ]:
df.head()

In [ ]:
## develop function for 

gb = df.drop_duplicates(['TimePoint', 'Action']).groupby(['Provider', 'enc_nbr', 'day'])
gb.groups
nodups = gb.get_group(('N.A.MD', '45770953', 27)).drop_duplicates(['TimePoint', 'Action'])
nodups.sort_values('TimePoint', inplace=True )
nodups['x'] = nodups.assign(ones=1)['ones'].cumsum()
nodups

#-----------------

gb = df\
        .drop_duplicates(['TimePoint', 'Action'])\
        .groupby(['Provider', 'enc_nbr', 'day'])

def x(g):
    nodups = g.drop_duplicates(['TimePoint', 'Action'])
    nodups.sort_values('TimePoint', inplace=True )
    nodups['x'] = nodups.assign(ones=1)['ones'].cumsum()
    return nodups

encLev = gb.apply(x)

In [ ]:
LookupProvider = pd.Series(index = encLev['Provider'].drop_duplicates())
LookupProvider[LookupProvider.isnull()] = range(len(LookupProvider))
LookupProvider = LookupProvider.astype(int)

LookupEnc = pd.Series(index = encLev['enc_nbr'].drop_duplicates())
LookupEnc[LookupEnc.isnull()] = range(len(LookupEnc))
LookupEnc = LookupEnc.astype(int)

In [ ]:
encLev['y'] = encLev['Provider'].map(LookupProvider)
encLev['y2'] = encLev['enc_nbr'].map(LookupEnc)

In [ ]:
#encLev

In [ ]:
def plot_points(df, shape):

    p = figure(plot_width=1600, plot_height=1600, 
                   title="xxxx", 
                   toolbar_location='right', tools="crosshair,pan,reset,save,wheel_zoom") 
                ## could add x_axis_type='datetime'
    
    NColors = df.Action.nunique()
    index_cmap = factor_cmap('Action', palette=palettes.viridis(NColors), factors=sorted(df.Action.unique()), end=1)

    def plot_actions(df, shape=shape):
        t = df.copy()
        #i = 1
        #t = df.assign(y = i)
        #print(t.shape, t['TimePoint'].max())
        #print(t.enc_nbr.nunique())
        #print(t.Action.nunique())
        
        if shape=='square':
            source = ColumnDataSource(df)

            p.square(x='x', y='y2',size=4, source=source,
                     color= index_cmap, 
                     hover_line_color="darkgrey", hover_fill_color=index_cmap, fill_alpha=1, legend='Action')
        
        elif shape=='quad':
            t['bottom'] = t['y2'] - .5
            t['right'] = t['x'] + 1
            source = ColumnDataSource(t)
            p.quad(top='y2', bottom='bottom', left='x', right='right', source=source,
                 color= index_cmap, 
                 hover_line_color="darkgrey", hover_fill_color=index_cmap, fill_alpha=1, legend='Action')
        
        ## HoverTools
        #p.add_tools(HoverTool(tooltips=[("Provider", "@Provider"),
        #                                ("Encounter", "@enc_nbr"),
        #                                ("Action", "@Action")],
        #                     mode='mouse'))

        #p.x_range=Range1d( time_point_min, time_point_max + pd.Timedelta(days=1))
        
    dft = plot_actions(df)        
    show(p)

In [ ]:
def plot_points(df, shape, plot_width=1600, plot_height=1800):

    p = figure(plot_width=plot_width, plot_height=plot_height, 
                   title="xxxx", 
                   toolbar_location='right', tools="crosshair,pan,reset,save,wheel_zoom") 
                ## could add x_axis_type='datetime'
    
    NColors = df.Action.nunique()
    index_cmap = factor_cmap('Action', palette=palettes.viridis(NColors), factors=sorted(df.Action.unique()), end=1)

    def plot_actions(df, shape=shape):
        t = df.copy()

        #i = 1
        #t = df.assign(y = i)
        #print(t.shape, t['TimePoint'].max())
        #print(t.enc_nbr.nunique())
        #print(t.Action.nunique())
        
        if shape=='square':
            source = ColumnDataSource(df)

            p.square(x='x', y='y2',size=4, source=source,
                     color= index_cmap, 
                     hover_line_color="darkgrey", hover_fill_color=index_cmap, fill_alpha=1, legend='Action')

        
        elif shape=='quad':
            t['bottom'] = t['y2'] - .5
            t['right'] = t['x'] + 1
            source = ColumnDataSource(t)
            p.quad(top='y2', bottom='bottom', left='x', right='right', source=source,
                 color= index_cmap, 
                 hover_line_color="darkgrey", hover_fill_color=index_cmap, fill_alpha=1, legend='Action')
        
        ## HoverTools
        #p.add_tools(HoverTool(tooltips=[("Provider", "@Provider"),
        #                                ("Encounter", "@enc_nbr"),
        #                                ("Action", "@Action")],
        #                     mode='mouse'))

        #p.x_range=Range1d( time_point_min, time_point_max + pd.Timedelta(days=1))
        
  
    ## anno
    def get_mean_y_points_for_anno():
        s = encLev[mm].reset_index(drop=True)[['Provider', 'enc_nbr']].drop_duplicates().reset_index(drop=True).reset_index().groupby('Provider')['index'].mean()
        s = s.to_frame().assign(x_anno=150).rename(columns={'index':'y_anno'}).reset_index()
        return s
    
    def get_mean_y_points_for_anno():
        s = encLev[mm].reset_index(drop=True)[['Provider', 'y2']].drop_duplicates().groupby('Provider')['y2'].mean()
        s = s.to_frame().assign(x_anno=150).rename(columns={'y2':'y_anno'}).reset_index()
        return s
    
    anno = bokeh_models.LabelSet(source=ColumnDataSource(get_mean_y_points_for_anno()), x='x_anno', y='y_anno', text='Provider', text_font_size="8pt", angle=0,  angle_units='deg')
    p.add_layout(anno)
        
    dft = plot_actions(df) 
    show(p)

In [ ]:
## keep only the records that have more then one template
m = encLev.groupby('y2').apply(lambda g: g['x'].nunique()) > 1
mm = encLev['y2'].map(m)

In [ ]:
encLev[mm].groupby('Provider').agg({'y2':['min', 'max']}).sort_values([('y2', 'min')], ascending=False)

In [ ]:
plot_points(encLev[mm], shape='quad')

In [ ]:
LookupProvider